In [1]:
import pandas as pd
import os
import numpy as np
import glob
from sklearn import preprocessing

# Audio feature extraction without time series

### Data extraction

In [2]:
path = "/Users/jiachenyao/Desktop/Thesis/OpenSmile/mix_emotions/"

# List all the .csv files in the folder
mix_emo_files = glob.glob(path + "/*.csv")

mix_emo_dfs = []
num_deleted = 0

for filename in mix_emo_files:
    df = pd.read_csv(filename, sep=";", index_col=None, header=0)
    
    df.drop(columns=["name","frameTime"], inplace=True)

    
            
    mix_emo_dfs.append(df)
        
print(len(mix_emo_dfs))

1080


In [3]:
#Extract functional features
mix_emo_functional_features_results = []

for df, file in zip(mix_emo_dfs, mix_emo_files):

    # Extract filename from filepath
    filename = os.path.basename(file)
    #print(filename)

    # Calculate functional features
    means = df.mean()
    
    result = pd.concat([means], axis=1)    

    # Rename columns with filename as prefix
    prefix = filename[:-4] # remove ".csv" extension
    result.columns = [f"{prefix}_mean"]


    mix_emo_functional_features_results.append(result)
print(len(mix_emo_functional_features_results))

1080


In [5]:
# Concate results to a single dataframe
mix_emo_audio_no_time_series_output = pd.concat(mix_emo_functional_features_results, axis=1)
mix_emo_audio_no_time_series_output = mix_emo_audio_no_time_series_output.T
mix_emo_audio_no_time_series_output = mix_emo_audio_no_time_series_output.reset_index().rename(columns={'index': 'dataname'})
mix_emo_audio_no_time_series_output.fillna(0,inplace=True)

In [6]:
mix_emo_audio_no_time_series_output

,dataname,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,A323_mix_disg_fea_5050_mean,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,-0.002506,0.011448,0.008568,0.0,0.0,0.0,0.0,0.0,0.0,-70.386313
1,A207_mix_ang_fea_3070_mean,14.601971,0.001505,14.567939,14.601971,14.636004,0.068065,0.004002,0.0,-0.052719,...,-0.003328,-0.002171,0.004820,0.0,0.0,0.0,0.0,0.0,0.0,-67.619258
2,A404_mix_ang_fea_5050_mean,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,-0.006813,0.018730,0.002318,0.0,0.0,0.0,0.0,0.0,0.0,-80.820458
3,A411_mix_fea_hap_3070_mean,9.823458,0.000939,9.804757,9.823458,9.842160,0.037402,0.004067,0.0,-0.027101,...,0.016270,-0.005100,0.007171,0.0,0.0,0.0,0.0,0.0,0.0,-64.655591
4,A417_mix_fea_hap_7030_mean,7.759801,0.002871,7.712825,7.759801,7.806778,0.093952,0.012649,0.0,-0.065644,...,0.011618,0.006881,0.007611,0.0,0.0,0.0,0.0,0.0,0.0,-69.966926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,A417_mix_hap_sad_5050_mean,1.061821,0.000397,1.054453,1.061821,1.069189,0.014736,0.000000,0.0,-0.012280,...,-0.000163,0.005474,0.002941,0.0,0.0,0.0,0.0,0.0,0.0,-83.424406
1076,A102_mix_ang_disg_3070_mean,14.564649,0.004108,14.469879,14.564649,14.659420,0.189541,0.087589,0.0,-0.070362,...,-0.007825,0.003711,0.002746,0.0,0.0,0.0,0.0,0.0,0.0,-68.497332
1077,A55_mix_fea_sad_5050_mean,1.155659,0.000065,1.154086,1.155659,1.157232,0.003146,0.002522,0.0,-0.000100,...,-0.001470,0.003126,0.004667,0.0,0.0,0.0,0.0,0.0,0.0,-77.598857
1078,A67_mix_disg_hap_5050_mean,3.461660,0.001094,3.443742,3.461660,3.479579,0.035837,0.003492,0.0,-0.026372,...,0.014840,0.005859,0.007789,0.0,0.0,0.0,0.0,0.0,0.0,-74.998171


### Preprocessing

In [7]:
# ratio
mix_emo_audio_no_time_ratio = mix_emo_audio_no_time_series_output.copy()
mix_emo_audio_no_time_ratio["dataname"] = mix_emo_audio_no_time_ratio["dataname"].str.extract('(mix.*)', expand=False)
mix_emo_audio_no_time_ratio["dataname"] = mix_emo_audio_no_time_ratio["dataname"].str.rsplit('_', 1).str[0]



In [9]:
# no ratio
mix_emo_audio_no_time_noratio = mix_emo_audio_no_time_series_output.copy()
mix_emo_audio_no_time_noratio["dataname"]= mix_emo_audio_no_time_noratio["dataname"].str.extract('(mix.*)', expand=False)
mix_emo_audio_no_time_noratio["dataname"] = mix_emo_audio_no_time_noratio["dataname"].apply(lambda x: '_'.join(x.split('_')[:-2]))

In [11]:
# min-max normalization

# ratio
cols_to_normalize_ratio = mix_emo_audio_no_time_ratio.columns.difference(['dataname'])
mix_emo_audio_no_time_ratio[cols_to_normalize_ratio] = mix_emo_audio_no_time_ratio[cols_to_normalize_ratio].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

# no ratio
cols_to_normalize_noratio = mix_emo_audio_no_time_noratio.columns.difference(['dataname'])
mix_emo_audio_no_time_noratio[cols_to_normalize_noratio] = mix_emo_audio_no_time_noratio[cols_to_normalize_noratio].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [15]:
# drop columns with all 0s
mix_emo_audio_no_time_ratio = mix_emo_audio_no_time_ratio.dropna(axis=1, how='all')
mix_emo_audio_no_time_noratio = mix_emo_audio_no_time_noratio.dropna(axis=1, how='all')

In [16]:
#save preprocessed files
mix_emo_audio_no_time_ratio.to_csv("/Users/jiachenyao/Desktop/Thesis/Preprocessed_data/mix_emo_audio_no_time_ratio.csv", index=False)

mix_emo_audio_no_time_noratio.to_csv("/Users/jiachenyao/Desktop/Thesis/Preprocessed_data/mix_emo_audio_no_time_noratio.csv", index=False)